In [4]:
# import pkg_resources
# import pip
# installedPackages = {pkg.key for pkg in pkg_resources.working_set}
# required = { 'openai','datasets', 'sklearn', 'tqdm'}
# missing = required - installedPackages
# if missing:
#     !pip install openai
#     !pip install datasets
#     !pip install scikit-learn
#     !pip install tqdm


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import openai
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix
from tqdm import tqdm


In [ ]:
#set your OpenAI API key
openai.api_key='your api key'

In [8]:
#load dataset and add prompt
def get_dataset(n_limit=0):
    dataset=load_dataset('financial_phrasebank','sentences_50agree')

    dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
    dataset = dataset["test"]

    if n_limit>0:
        dataset['train']=dataset['train'][:n_limit]
    print("size of dataset: ", len( dataset['train']['sentence']))
    text_inputs = dataset['train']['sentence']
    process_inputs=[
        f"Human: Determine the sentiment of the financial news as negative, neutral or positive: {text_inputs[i]} Assistant: "
        for i in range(len(text_inputs))]
    labels=dataset['train']['label']
    return process_inputs, labels    

In [10]:
#ChatGPT API
def chat_with_gpt(prompt):
    response=openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role":"system", "content":"Hello"},
            {"role": "user","content":prompt}
        ]
    )
    return response.choices[0].message.content.strip()

In [21]:
sentences, labels = get_dataset()

Found cached dataset financial_phrasebank (/Users/bruce/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 236.91it/s]

size of dataset:  4846


In [22]:
len(sentences)

4846

In [23]:
senteces[0]

'Human: Determine the sentiment of the financial news as negative, neutral or positive: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing . Assistant: '

In [24]:
#positive: 2, negative: 0, neutral:1
labels[0]

1

In [36]:
#chatgpt response 
preds=[]

In [37]:
#test run
import time
for prompt, label in tqdm(zip(sentences[0:5],labels)):
    #print(prompt, label)
    time.sleep(1)
    response=chat_with_gpt(prompt)
    if   "negative" in response:
        preds.append(0)
    elif "neutral" in response:
        preds.append(1)
    elif "positive" in response:
        preds.append(2)
    else:
        preds.append(1)

5it [00:12,  2.45s/it]


In [42]:
response

'Neutral'

In [38]:
len(preds)

5

In [39]:
preds

[1, 1, 0, 2, 1]

In [40]:
print(f"Accuracy: {accuracy_score(labels[0:5],preds[0:5])}")

Accuracy: 0.8


In [41]:
print(f"F1: {f1_score(labels[0:5],preds[0:5],average='macro')}")

F1: 0.8222222222222223
